In [2]:
from jupyter_dash import JupyterDash
from dash import html, dcc
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import requests
import json
import datetime

# Initialize the JupyterDash app
app = JupyterDash(__name__, 
                  external_stylesheets=['https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.3.0/css/bootstrap.min.css'])

# Layout for the Dash app
app.layout = html.Div([
    html.H1("Product Price Analysis Dashboard", style={'textAlign': 'center'}),
    
    # Dropdown to select product
    dcc.Dropdown(id='product-dropdown', options=[], value=None, placeholder="Select a product"),
    
    # Product Info
    html.Div(id='product-info'),
    
    # Price Comparison Chart
    dcc.Graph(id='price-comparison'),
    
    # Price History Chart
    dcc.Graph(id='price-history-chart'),
    
    # Statistics
    html.Div(id='product-statistics')
])

# Fetch product list and populate dropdown
@app.callback(
    Output('product-dropdown', 'options'),
    Input('product-dropdown', 'value')
)
def fetch_products(selected_value):
    response = requests.get("http://127.0.0.1:8000/products?limit=10")
    products = response.json()
    return [{'label': product['name'], 'value': product['sku']} for product in products]

# Update product info, price comparison, and history chart
@app.callback(
    [Output('product-info', 'children'),
     Output('price-comparison', 'figure'),
     Output('price-history-chart', 'figure'),
     Output('product-statistics', 'children')],
    Input('product-dropdown', 'value')
)
def update_product_info(selected_sku):
    if not selected_sku:
        return "", {}, {}, ""
    
    # Fetch product details
    response = requests.get(f"http://127.0.0.1:8000/product/{selected_sku}")
    product = response.json()
    
    # Product Info
    product_info = html.Div([
        html.H4(product['name']),
        html.P(f"Price: {product['price']} VND"),
        html.P(f"Original Price: {product['original_price']} VND"),
        html.P(f"List Price: {product['list_price']} VND"),
        html.P(f"SKU: {product['sku']}"),
        html.P(f"Seller: {product['current_seller']}"),
    ])
    
    # Price History Chart
    price_history = product.get('price_history', [])
    if price_history:
        price_dates = [entry['date'] for entry in price_history]
        prices = [entry['price'] for entry in price_history]
        
        price_history_fig = {
            'data': [go.Scatter(x=price_dates, y=prices, mode='lines', name='Price History')],
            'layout': go.Layout(title='Price History', xaxis={'title': 'Date'}, yaxis={'title': 'Price (VND)'}),
        }
    else:
        price_history_fig = {}

    # Statistics
    statistics_response = requests.get(f"http://127.0.0.1:8000/product/{selected_sku}/statistics")
    statistics = statistics_response.json()
    statistics_info = html.Div([
        html.P(f"Average Price: {statistics['average_price']} VND"),
        html.P(f"Min Price: {statistics['min_price']} VND"),
        html.P(f"Max Price: {statistics['max_price']} VND"),
    ])
    
    return product_info, {}, price_history_fig, statistics_info

# Run the app
if __name__ == '__main__':
    app.run_server(mode='external', debug=True, port=8050)


D:\FIA1471\.venv\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/


In [ ]:
!uvicorn backend:app --reload --host 0.0.0.0 --port 8000

In [ ]:
import requests

url = "http://localhost:8000/products?limit=10&skip=0"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    print(response.json())  # Print the response as a JSON object
else:
    print(f"Error {response.status_code}: {response.text}")